In [241]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mutual_info_score
from sklearn.model_selection import train_test_split

In [242]:
%matplotlib inline

In [243]:
df = pd.read_csv('https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv')

In [244]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


**Preparing the dataset**


- Check if the missing values are presented in the features.
- If there are missing values:
    - For caterogiral features, replace them with 'NA'
    - For numerical features, replace with with 0.0

In [245]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [246]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [247]:
categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)
categorical_columns

['lead_source', 'industry', 'employment_status', 'location']

In [248]:
df[categorical_columns] = df[categorical_columns].fillna('NA')

In [249]:
numerical_columns = list(df.dtypes[df.dtypes != 'object'].index)
numerical_columns

['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score',
 'converted']

In [250]:
df[numerical_columns] = df[numerical_columns].fillna(0.0)

In [251]:
df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

# Q1.

What is the most frequent observation (mode) for the column industry?

-    NA
-    technology
-    healthcare
-    retail

In [252]:
df['industry'].value_counts()

industry
retail           203
finance          200
other            198
healthcare       187
education        187
technology       179
manufacturing    174
NA               134
Name: count, dtype: int64

# Q2.

Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features.

What are the two features that have the biggest correlation?

-    interaction_count and lead_score
-    number_of_courses_viewed and lead_score
-    number_of_courses_viewed and interaction_count
-    annual_income and interaction_count

Only consider the pairs above when answering this question.

In [253]:
corr = df[['interaction_count', 'lead_score', 'number_of_courses_viewed', 'annual_income']].corr()
corr

,interaction_count,lead_score,number_of_courses_viewed,annual_income
interaction_count,1.000000,0.009888,-0.023565,0.027036
lead_score,0.009888,1.000000,-0.004879,0.015610
number_of_courses_viewed,-0.023565,-0.004879,1.000000,0.009770
annual_income,0.027036,0.015610,0.009770,1.000000


In [254]:
abs(corr).unstack().sort_values(ascending=False)

interaction_count         interaction_count           1.000000
lead_score                lead_score                  1.000000
annual_income             annual_income               1.000000
number_of_courses_viewed  number_of_courses_viewed    1.000000
interaction_count         annual_income               0.027036
annual_income             interaction_count           0.027036
number_of_courses_viewed  interaction_count           0.023565
interaction_count         number_of_courses_viewed    0.023565
annual_income             lead_score                  0.015610
lead_score                annual_income               0.015610
interaction_count         lead_score                  0.009888
lead_score                interaction_count           0.009888
annual_income             number_of_courses_viewed    0.009770
number_of_courses_viewed  annual_income               0.009770
lead_score                number_of_courses_viewed    0.004879
number_of_courses_viewed  lead_score                  0

**Split the data**

-    Split your data in train/val/test sets with 60%/20%/20% distribution.
-    Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
-    Make sure that the target value y is not in your dataframe.

In [255]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [256]:
len(df_train), len(df_val), len(df_test)

(876, 293, 293)

In [257]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [258]:
y_train = df_train['converted'].values
y_val = df_val['converted'].values
y_test = df_test['converted'].values

del df_train['converted']
del df_val['converted']
del df_test['converted']

# Q3.

-    Calculate the mutual information score between y and other categorical variables in the dataset. Use the training set only.
-    Round the scores to 2 decimals using round(score, 2).

Which of these variables has the biggest mutual information score?

-    industry
-    location
-    lead_source
-    employment_status

In [259]:
def mutual_info_churn_score(series):
    return mutual_info_score(series, y_train)

In [260]:
mi = df_train[categorical_columns].apply(mutual_info_churn_score)
mi.sort_values(ascending=False)

lead_source          0.035396
employment_status    0.012938
industry             0.011575
location             0.004464
dtype: float64

# Q4.

-    Now let's train a logistic regression.
-    Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
-    Fit the model on the training dataset.
    -        To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
    -        model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
-    Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

What accuracy did you get?

-    0.64
-    0.74
-    0.84
-    0.94

In [261]:
df_train.columns

Index(['lead_source', 'industry', 'number_of_courses_viewed', 'annual_income',
       'employment_status', 'location', 'interaction_count', 'lead_score'],
      dtype='object')

In [262]:
categorical_columns

['lead_source', 'industry', 'employment_status', 'location']

In [263]:
numerical_columns

['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score',
 'converted']

In [264]:
categorical_columns = ['lead_source', 'industry', 'employment_status', 'location']

In [265]:
# drop target
numerical_columns = ['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score']

In [266]:
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical_columns + numerical_columns].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical_columns + numerical_columns].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [267]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [268]:
model.intercept_[0]

np.float64(-0.06914728027832363)

In [269]:
model.coef_[0].round(3)

array([-0.   , -0.015,  0.034,  0.003,  0.012, -0.103, -0.025,  0.049,
       -0.02 , -0.013, -0.003, -0.009, -0.032, -0.016,  0.311,  0.051,
        0.02 , -0.012, -0.012, -0.115,  0.08 , -0.03 ,  0.004, -0.011,
       -0.011, -0.006,  0.008,  0.006, -0.033, -0.025,  0.454])

In [270]:
y_pred = model.predict_proba(X_val)[:, 1]

In [271]:
converted_decision = (y_pred >= 0.5)

In [272]:
base_acc = round((y_val == converted_decision).mean(), 2)
base_acc

np.float64(0.7)

# Q5.

-    Let's find the least useful feature using the feature elimination technique.
-    Train a model using the same features and parameters as in Q4 (without rounding).
-    Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
-    For each feature, calculate the difference between the original accuracy and the accuracy without the feature.

Which of following feature has the smallest difference?

-    'industry'
-    'employment_status'
-    'lead_score'

    Note: The difference doesn't have to be positive.

In [273]:
for f in numerical_columns:
    subset = [c for c in (categorical_columns + numerical_columns) if c != f]
    
    dv = DictVectorizer(sparse=False)
    train_dict = df_train[subset].to_dict(orient='records')
    val_dict = df_val[subset].to_dict(orient='records')
    X_train = dv.fit_transform(train_dict)
    X_val = dv.transform(val_dict)
    
    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    
    y_pred = model.predict_proba(X_val)[:, 1]
    converted_decision = (y_pred >= 0.5)
    acc = (y_val == converted_decision).mean()
    
    print(f"{f} diff = {round(base_acc - acc, 2)}")

number_of_courses_viewed diff = 0.14
annual_income diff = -0.15
interaction_count diff = 0.14
lead_score diff = -0.01


# Q6.

-    Now let's train a regularized logistic regression.
-    Let's try the following values of the parameter C: [0.01, 0.1, 1, 10, 100].
-    Train models using all the features as in Q4.
-    Calculate the accuracy on the validation dataset and round it to 3 decimal digits.

Which of these C leads to the best accuracy on the validation set?

-    0.01
-    0.1
-    1
-    10
-    100

    Note: If there are multiple options, select the smallest C.

In [274]:
for c in [0.01, 0.1, 1, 10, 100]:
    model = LogisticRegression(solver='liblinear', C=c, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    
    y_pred = model.predict_proba(X_val)[:, 1]
    converted_decision = (y_pred >= 0.5)
    reg_acc = (y_val == converted_decision).mean()
    
    print(f"C={c}, accuracy={round(reg_acc, 4)}")

C=0.01, accuracy=0.6962
C=0.1, accuracy=0.6997
C=1, accuracy=0.7065
C=10, accuracy=0.7065
C=100, accuracy=0.7065
